In [8]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
from plotnine import *

# 0. Together

Interpreting coefficients:

Interpret the following coefficients.


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import accuracy_score, confusion_matrix,\
 f1_score, recall_score, precision_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv

In [10]:
d = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Lizzo_data.csv")

# turn boolean into 0's and 1's (not necessary, but helps with confusion over True/False)

d["explicit"] = d["explicit"].astype("int")
d.head()

,Unnamed: 0,artist_name,artist_id,album_id,album_type,album_release_date,album_release_year,album_release_date_precision,danceability,energy,...,track_name,track_preview_url,track_number,type,track_uri,external_urls.spotify,album_name,key_name,mode_name,key_mode
0,1,Lizzo,56oDRnqbIiwx4mymNEv7dS,6dFFcYQ8VhifgdKgYY5LYL,album,2019-04-19,2019,day,0.566,0.660,...,Cuz I Love You,https://p.scdn.co/mp3-preview/04472d81bc5a9669...,1,track,spotify:track:6YdQgWSpsxhVeX6Xmv3IFJ,https://open.spotify.com/track/6YdQgWSpsxhVeX6...,Cuz I Love You,C#,major,C# major
1,2,Lizzo,56oDRnqbIiwx4mymNEv7dS,6dFFcYQ8VhifgdKgYY5LYL,album,2019-04-19,2019,day,0.694,0.807,...,Like a Girl,https://p.scdn.co/mp3-preview/4f917112d7f75e2c...,2,track,spotify:track:5I7sGubUsKo4mVJpBoSVUr,https://open.spotify.com/track/5I7sGubUsKo4mVJ...,Cuz I Love You,D,major,D major
2,3,Lizzo,56oDRnqbIiwx4mymNEv7dS,6dFFcYQ8VhifgdKgYY5LYL,album,2019-04-19,2019,day,0.767,0.889,...,Juice,https://p.scdn.co/mp3-preview/7ca1598cec36c491...,3,track,spotify:track:0k664IuFwVP557Gnx7RhIl,https://open.spotify.com/track/0k664IuFwVP557G...,Cuz I Love You,G,major,G major
3,4,Lizzo,56oDRnqbIiwx4mymNEv7dS,6dFFcYQ8VhifgdKgYY5LYL,album,2019-04-19,2019,day,0.693,0.849,...,Soulmate,https://p.scdn.co/mp3-preview/467bc6a734bfb66e...,4,track,spotify:track:6h2wpo2pshM8QnAvRySEO0,https://open.spotify.com/track/6h2wpo2pshM8QnA...,Cuz I Love You,G,major,G major
4,5,Lizzo,56oDRnqbIiwx4mymNEv7dS,6dFFcYQ8VhifgdKgYY5LYL,album,2019-04-19,2019,day,0.674,0.542,...,Jerome,https://p.scdn.co/mp3-preview/8f662de51ad3f613...,5,track,spotify:track:3kxsEF30mM0TZWfkOv4XsS,https://open.spotify.com/track/3kxsEF30mM0TZWf...,Cuz I Love You,G#,major,G# major


In [11]:
predictors = ["danceability", "energy", "instrumentalness", "explicit"]
cont = ["danceability", "energy", "instrumentalness"]

X = d[predictors]
y = d["mode"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state = 123)
#random_state makes sure we have the same split each time
X.head()

,danceability,energy,instrumentalness,explicit
0,0.566,0.660,0.00000,1
1,0.694,0.807,0.00000,1
2,0.767,0.889,0.00000,1
3,0.693,0.849,0.00127,1
4,0.674,0.542,0.00000,0


In [12]:
# z
z = StandardScaler()
X_train[cont] = z.fit_transform(X_train[cont])
X_test[cont] = z.transform(X_test[cont])

X_train.head()

,danceability,energy,instrumentalness,explicit
1,0.033718,0.655584,-0.370893,1
6,1.139904,0.024802,-0.330559,1
27,0.264505,-0.680191,-0.370893,0
35,1.498022,1.687099,-0.370893,1
12,0.049634,0.700110,-0.370893,0


In [13]:

# model
lr = LogisticRegression()

# fit
lr.fit(X_train,y_train)

coef = pd.DataFrame({"coefNames": predictors, "logOdds": lr.coef_[0],
                    "Odds": np.exp(lr.coef_[0])})
coef

#why don't I have probabilities here?

,coefNames,logOdds,Odds
0,danceability,-0.311998,0.731983
1,energy,0.291403,1.338304
2,instrumentalness,-0.079619,0.923468
3,explicit,-0.529251,0.589046


# 1. The Math of Log Odds, Odds, and Probabilities

In the lecture, you saw a beautiful (if I do say so myself...) graph demonstrating the way that Log Odds, Odds, and Probabilities change as a result of a 1-unit change in a predictor variable.

Now YOU'RE going to make your own version to help you gain an intuitive sense for the math of log odds, odds, and probabilities. 

We're going to write a function that simulates a super simple logistic relationship: 1 predictor (X) and 1 binary outcome (y). 

- Add code to calculate the odds and probabilities for each data point based on the log odds and predictor values

(if it helps, think of this as predicting whether or not someone is registered to vote based on their age. **OH SPEAKING OF**: are you [registered to vote](https://www.usa.gov/register-to-vote)?)



Use the dataframe that calling `LogisticDataSimulator()` with an intercept of 1 and slope of 0.5 makes, and recreate the graph of the log odds, odds, and probabilities from the [lecture](https://github.com/cmparlettpelleriti/CPSC392ParlettPelleriti/blob/master/InClass/LectureSlides/Lecture%208%20and%209%20_%20Logistic%20Regression.pdf). Make separate graphs for logodds, odds, and probabilities to make your life easier.

I don't believe that plotnine supports subtitles, so no need to include those but include the colored lines, the titles/axis labels, and the dotted vertical lines! Don't be afraid to google how to do things in ggplot (hint: google `geom_line()` and `geom_vline()`).

In [ ]:
def LogisticDataSimulator(intercept, slope, limits = 5):
    
    #generate 1000 evenly spaced values between -limits and limits
    predictor = np.linspace(-limits, limits, 1000)
    
    # log odds of being registered to vote
    log_odds = intercept + slope*predictor
    
    
    # odds of being registered to vote
    #################################
    # HINT: maybe look at Class 8 Classwork...
    odds = ???
    
    #################################
    
    # probability of being registered to vote
    #################################
    # HINT: maybe look at Class 8 Classwork...
    probabilities =  ???
    
    #################################
    
    # put into a dataframe
    df = pd.DataFrame({"x": predictor,
                      "logodds": log_odds,
                       "odds": odds,
                       "probabilities": probabilities})
    
    return(df)

In [14]:
#################################
### YOUR CODE HERE ###
#################################

### *Question*

Describe in detail how the graphs demonstrate what we learned in class: that the change in predicted probability in response to a 1 unit increase of our predictor is *not constant*?

<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" width = 200px />


### YOUR ANSWER HERE 